In [100]:
import networkx as nx
import numpy as np
import pandas as pd
import seaborn as sns
from layout import Layout

In [101]:
countries = pd.read_csv("mig_nodes.csv", index_col=0)
migration = pd.read_csv("mig_edges.csv")

In [102]:
migration["total"] = migration[["migration_1990_total","migration_1995_total","migration_2000_total","migration_2005_total","migration_2010_total","migration_2015_total"]].sum(axis=1)
migration = migration[migration["total"] != 0]

In [103]:
migration

,source,target,migration_1990_total,migration_1990_male,migration_1990_female,migration_1995_total,migration_1995_male,migration_1995_female,migration_2000_total,migration_2000_male,...,migration_2005_total,migration_2005_male,migration_2005_female,migration_2010_total,migration_2010_male,migration_2010_female,migration_2015_total,migration_2015_male,migration_2015_female,total
0,0,64,237,194,43,188,150,38,154,117,...,183,142,41,198,152,46,235,173,62,1195
1,0,118,677,556,121,452,351,101,236,155,...,259,187,72,284,206,78,320,231,89,2228
2,0,142,64,26,38,61,25,36,58,23,...,45,18,27,43,18,25,39,17,22,310
3,0,191,59,37,22,36,24,12,30,20,...,47,27,20,69,41,28,83,50,33,324
4,0,202,8485,3726,4759,12248,5518,6730,17118,7874,...,12636,5687,6949,8900,3906,4994,7587,3279,4308,66974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11223,231,220,4767,2148,2619,7571,3451,4120,10686,4909,...,26585,12282,14303,42811,19381,23430,50001,22556,27445,142421
11224,231,11,8265,4184,4081,10498,5291,5207,12110,6110,...,21690,10960,10730,33720,16870,16850,38843,19418,19425,125126
11225,231,147,676,323,353,822,398,424,2807,1365,...,7794,3783,4011,8170,3959,4211,8416,4070,4346,28685
11226,231,68,0,0,0,0,0,0,0,0,...,0,0,0,6,6,0,6,6,0,12


In [104]:
G_tot = nx.DiGraph()
for _, row in migration.iterrows():
    G_tot.add_edge(row["source"], row["target"], weight=row["total"])
nx.info(G_tot)

'DiGraph with 232 nodes and 11159 edges'

In [105]:
G_1990 = nx.DiGraph()
for _, row in migration.iterrows():
    G_1990.add_edge(row["source"], row["target"], weight=row["migration_1990_total"])
nx.info(G_1990)

'DiGraph with 232 nodes and 11159 edges'

In [106]:
G_2015 = nx.DiGraph()
for _, row in migration.iterrows():
    G_2015.add_edge(row["source"], row["target"], weight=row["migration_2015_total"])
nx.info(G_2015)

'DiGraph with 232 nodes and 11159 edges'

In [107]:
pos = nx.spring_layout(G_tot, dim=3)

In [108]:
layout_tot = Layout(G_tot, pos, node_names=countries["name"])
layout_1990 = Layout(G_1990, pos, node_names=countries["name"])
layout_2015 = Layout(G_2015, pos, node_names=countries["name"])
layout_2015_comm = Layout(G_2015, pos, node_names=countries["name"])

In [109]:
region_colors = {
    'Africa': (0, 0, 0, 100),
    'Asia': (252, 177, 49, 100),
    'Europe': (0, 129, 200, 100),
    'Latin America and the Caribbean': (238, 51, 78, 100),
    'Northern America': (156, 39, 176, 100),
    'Oceania': (0, 166, 81, 100)
}

def continent_node_color(node):
    region = countries.loc[node, "region"]
    return region_colors[region]

continent_node_colors = [continent_node_color(node) for node in G_tot]
layout_1990.node_colors = continent_node_colors
layout_2015.node_colors = continent_node_colors

In [110]:
communities = nx.algorithms.community.greedy_modularity_communities(G_2015, weight="weight")
n_comms = len(communities)
colors = 100 * np.ones((n_comms, 4))
colors[:,:3] = np.array(sns.color_palette(n_colors=n_comms)) * 255
colors = colors.astype(int)

comm_node_colors = np.zeros((len(G_2015), 4))
for i, community in enumerate(communities):
    for node in community:
        comm_node_colors[node] = colors[i]

layout_2015_comm.node_colors = comm_node_colors

In [111]:
green = (56, 142, 60, 100)
red = (211, 47, 47, 100)
grey = (97, 97, 97, 100)

def net_node_color(G, node):
    in_deg = G.in_degree(node, weight="weight")
    out_deg = G.out_degree(node, weight="weight")
    net = in_deg - out_deg
    if net > 0:
        return green
    if net < 0:
        return red
    return grey

layout_tot.node_colors = [net_node_color(G_tot, node) for node in G_tot]

In [112]:
def edge_color(weight):
    if weight <= 0:
        glow = 0
    else:
        glow = 100 + 155 * np.log10(weight) / 7
    return (255, 255, 255, int(glow))

layout_tot.edge_colors = [edge_color(weight) for weight in migration["total"]]
layout_1990.edge_colors = [edge_color(weight) for weight in migration["migration_1990_total"]]
layout_2015.edge_colors = layout_2015_comm.edge_colors = [edge_color(weight) for weight in migration["migration_2015_total"]]

In [113]:
layout_tot.write(node_file_path="migration_total_nodes.csv", edge_file_path="migration_total_edges.csv")
layout_1990.write(node_file_path="migration_1990_nodes.csv", edge_file_path="migration_1990_edges.csv")
layout_2015.write(node_file_path="migration_2015_nodes.csv", edge_file_path="migration_2015_edges.csv")
layout_2015_comm.write(node_file_path="migration_2015_comm_nodes.csv", edge_file_path="migration_2015_comm_edges.csv")